# Agents

## Environment Setup

- LangChain: [v0.2.17](https://pypi.org/project/langchain/0.2.17/)
- Python: 3.12.7

In [1]:
import os

#load env vars
from dotenv import load_dotenv
load_dotenv(".env")

from langchain_openai import AzureChatOpenAI

os.environ["LANGCHAIN_TRACING_V2"] = "true"
LANGCHAIN_API_KEY = os.environ["LANGCHAIN_API_KEY"]

TAVILY_API_KEY = os.environ["TAVILY_API_KEY"]
Azure_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [2]:
import json

def pretty_json(data):
    return json.dumps(data, indent=4,sort_keys=True ,ensure_ascii=False)

### Test Azure Open API

In [3]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="How many Rs are there in strawberry?")])
response.content

'The word "strawberry" contains three Rs.'

## Define tools

### 我們選擇 Tavily

官方教學選定的工具 [Tavily](https://tavily.com/)： 讓 LLM 使用網路搜尋的服務

我們嘗試問一個有時效性的問題：`2024美國大選的結果？`

從輸出可以看到，Tavily搜尋了一些新聞網站，並且列出網站內的內容。

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("What's 2024 US presidential election result?")

print(pretty_json(search_results))

[
    {
        "content": "US Presidential Election Results 2024 - BBC News Close menu BBC News Kamala Harris of the Democrat party has 0 electoral college votes. Donald Trump of the Republican party has 0 electoral college votes. Kamala Harris of the Democrat party has 158,810 votes (38.4%) Donald Trump of the Republican party has 249,225 votes (60.2%) US presidential election results 2024 US election 2024 Voting in some states is particularly hard to predict, with polls showing they could be won by the Republicans or the Democratic party. The battleground states that could decide the 2024 presidential election are: Voters in 11 states will also elect a governor. US election polls: Who is ahead - Harris or Trump? US election 2024 About the BBC",
        "url": "https://www.bbc.com/news/election/2024/us/results"
    },
    {
        "content": "US Presidential Election Results 2024 - BBC News Close menu BBC News Kamala Harris of the Democrat party has 27 electoral college votes. Donal

### 將其綁定在model上

In [5]:
# 我們已定義：search = TavilySearchResults(max_results=2)

tools = [search]
model_with_tools = model.bind_tools(tools)

### 嘗試呼叫

從我們的問題，`model_with_tools` 會根據問題的內容，選擇適合的工具，並且呼叫該工具。

In [6]:
# 此問題理應需要搜尋資料才能回答，可以看到它呼叫了工具

response = model_with_tools.invoke([HumanMessage(content="What's 2024 US presidential election result?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {pretty_json(response.tool_calls)}")

ContentString: 
ToolCalls: [
    {
        "args": {
            "query": "2024 US presidential election results"
        },
        "id": "call_7wnuGCmA9N2ESXcPZcfoIZSt",
        "name": "tavily_search_results_json",
        "type": "tool_call"
    }
]


In [7]:
# 不須使用 Search Tool 就能回答的問題，就不會呼叫

response = model_with_tools.invoke([HumanMessage(content="How many Rs are there in strawberry?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: There are three Rs in the word "strawberry."
ToolCalls: []


## Create the Agent

我們開始建立Agent，並且綁定了剛剛建立的 model(Azure OpenAI) 和 tools[Search]

註： 我們使用 LangGraph 來建立 Agent

In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

詢問不須使用 tool 的問題

In [9]:
from langchain.output_parsers import PydanticOutputParser


response = agent_executor.invoke(
    {"messages": [HumanMessage(content="hi!")]}
)

response["messages"]


[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='c99d606b-2559-4df8-9168-40049da6130d'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_04751d0b65', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'fi

詢問需要使用 tool 的問題

In [10]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)

response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='9397d773-9377-407a-b4a5-fc8972d10385'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3ehzoqSqwpcHgMB8W2kyC1Lx', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_04751d0b65', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprob

### 支援 Streaming

#### Streaming Message

使用 `stream()` 方法

In [11]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3ehzoqSqwpcHgMB8W2kyC1Lx', 'function': {'arguments': '{"query":"current weather San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 86, 'total_tokens': 107, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_04751d0b65', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-53a952cf-82d7-4a9b-8ccd-3114e5e42160-0', tool_calls=[{'name': 'tavily

#### Streaming Token

使用 `astream_events()` 方法

In [12]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1732455394, \'localtime\': \'2024-11-24 05:36\'}, \'current\': {\'last_updated_epoch\': 1732455000, \'last_updated\': \'2024-11-24 05:30\', \'temp_c\': 7.8, \'temp_f\': 46.0, \'is_day\': 0, \'condition\': {\'text\': \'Fog\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/248.png\', \'code\': 1135}, \'wind_mph\': 5.1, \'wind_kph\': 8.3, \'wind_degree\': 160, \'wind_dir\': \'SSE\', \'pressure_mb\': 1017.0, \'pressure_in\': 30.03, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 89, \'cloud\': 25, \'feelslike_c\': 6.4, \'feelslike_f\': 43.4, \'windchill_c\

## 增加 memory 功能

先前的 Agent 是無狀態的（stateless），我們可以加入 Thread_id ，來讀取先前的對話內容。

In [13]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

### memory 測試範例

#### 使用同一 Thread

能記憶同一對話紀錄，記得我叫做 Bob

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_04751d0b65', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 

In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 106, 'total_tokens': 119, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_04751d0b65', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-1309a8d8-ac78-4311-814d-4b4cb9112fba-0', usage_metadata={'input_tokens': 106, 'output_tokens': 13, 'total_tokens': 119, 'input_token_deta

##### 覆蓋新的 Thread

memory 就沒有我告訴它我叫 Bob 的記憶了

In [16]:
config = {"configurable": {"thread_id": "xyz123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't have access to personal information about you, including your name. If you'd like to share your name or any other details, feel free to do so!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 84, 'total_tokens': 117, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_04751d0b65', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered'